In [1]:
import secrets  
from sympy import randprime
import numpy as np

In [2]:
p = 97  
a = 2   
b = 3   

G = (3, 6)
O = None

In [3]:
def generate_finite_field():
    prime = randprime(100, 1000)
    return prime

In [4]:
def is_valid_curve(a, b, p):
    discriminant = (4 * a**3 + 27 * b**2) % p
    return discriminant != 0

In [5]:
def get_curve_equation_coeffs(p):
    a = np.random.randint(0, 100)
    b = np.random.randint(0, 100)
    while not is_valid_curve(a, b, p):
        a = np.random.randint(0, 100)
        b = np.random.randint(0, 100)
    return a, b

In [6]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [7]:
def is_on_curve(x, y):
    return ((y ** 2) - (x ** 3 + a * x + b))% p ==  0

In [8]:
def inverse_mod(k, p):
    if k == 0:
        raise ZeroDivisionError("Division by zero")
    return pow(k, p - 2, p)  

In [9]:
def point_add(P, Q):
    if P == O:
        return Q
    if Q == O:
        return P

    x1, y1 = P
    x2, y2 = Q

    if x1 == x2 and y1 != y2:
        return O  

    if P == Q:  
        m = (3 * x1 * x1 + a) * inverse_mod(2 * y1, p)
    else:  
        m = (y2 - y1) * inverse_mod(x2 - x1, p)

    m %= p
    x3 = (m * m - x1 - x2) % p
    y3 = (m * (x1 - x3) - y1) % p

    return (x3, y3)

In [10]:
def scalar_mult(k, P):
    result = O
    while k > 0:
        if k % 2 == 1:
            print(f"Adding {result} and {P}")
            result = point_add(result, P)
            print(f"Result after addition: {result}")
        P = point_add(P, P)
        print(f"Doubling point: {P}")
        k = k // 2
    return result

In [11]:
dummy_point = (2, 3)
scalar_mult(13, dummy_point)

Adding None and (2, 3)
Result after addition: (2, 3)
Doubling point: (23, 45)
Doubling point: (73, 53)
Adding (2, 3) and (73, 53)
Result after addition: (94, 62)
Doubling point: (32, 25)
Adding (94, 62) and (32, 25)
Result after addition: (65, 57)
Doubling point: (45, 41)


(65, 57)

In [12]:
def generate_keypair():
    private_key = secrets.randbelow(p - 1) + 1
    public_key = scalar_mult(private_key, G)
    return private_key, public_key

In [13]:
def encrypt(Pm, public_key):
    k = secrets.randbelow(p - 1) + 1
    C1 = scalar_mult(k, G)
    S = scalar_mult(k, public_key)
    C2 = point_add(Pm, S)
    return (C1, C2)

In [14]:
def decrypt(C1, C2, private_key):
    S = scalar_mult(private_key, C1)
    S_inv = (S[0], (-S[1]) % p)
    Pm = point_add(C2, S_inv)
    return Pm

In [15]:
Pm = (3, 6)  

private_key, public_key = generate_keypair()
print(f"Private Key: {private_key}")
print(f"Public Key: {public_key}")

C1, C2 = encrypt(Pm, public_key)
print(f"Encrypted: C1 = {C1}, C2 = {C2}")

decrypted = decrypt(C1, C2, private_key)
print(f"Decrypted: {decrypted}")

Doubling point: (80, 10)
Doubling point: (3, 91)
Adding None and (3, 91)
Result after addition: (3, 91)
Doubling point: (80, 87)
Adding (3, 91) and (80, 87)
Result after addition: (80, 10)
Doubling point: (3, 6)
Adding (80, 10) and (3, 6)
Result after addition: (80, 87)
Doubling point: (80, 10)
Doubling point: (3, 91)
Adding (80, 87) and (3, 91)
Result after addition: (80, 10)
Doubling point: (80, 87)
Private Key: 92
Public Key: (80, 10)
Adding None and (3, 6)
Result after addition: (3, 6)
Doubling point: (80, 10)
Doubling point: (3, 91)
Doubling point: (80, 87)
Doubling point: (3, 6)
Doubling point: (80, 10)
Adding (3, 6) and (80, 10)
Result after addition: (80, 87)
Doubling point: (3, 91)
Adding None and (80, 10)
Result after addition: (80, 10)
Doubling point: (3, 91)
Doubling point: (80, 87)
Doubling point: (3, 6)
Doubling point: (80, 10)
Doubling point: (3, 91)
Adding (80, 10) and (3, 91)
Result after addition: (3, 6)
Doubling point: (80, 87)
Encrypted: C1 = (80, 87), C2 = (80, 10)

In [16]:
# create test set of points on the curve
points_on_curve = []
for x in range(p):
    for y in range(p):
        if is_on_curve(x, y):
            points_on_curve.append((x, y))
            
print(f"Points on the curve: {points_on_curve}")
print(f"Total points on the curve: {len(points_on_curve)}")

Points on the curve: [(0, 10), (0, 87), (1, 43), (1, 54), (3, 6), (3, 91), (4, 47), (4, 50), (10, 21), (10, 76), (11, 17), (11, 80), (12, 3), (12, 94), (17, 10), (17, 87), (20, 34), (20, 63), (21, 24), (21, 73), (22, 5), (22, 92), (23, 24), (23, 73), (24, 2), (24, 95), (25, 35), (25, 62), (27, 7), (27, 90), (28, 34), (28, 63), (29, 43), (29, 54), (30, 0), (32, 7), (32, 90), (37, 22), (37, 75), (38, 7), (38, 90), (39, 6), (39, 91), (44, 20), (44, 77), (46, 25), (46, 72), (47, 18), (47, 79), (49, 34), (49, 63), (50, 19), (50, 78), (52, 29), (52, 68), (53, 24), (53, 73), (54, 12), (54, 85), (55, 6), (55, 91), (56, 8), (56, 89), (59, 32), (59, 65), (65, 32), (65, 65), (67, 43), (67, 54), (68, 0), (70, 32), (70, 65), (73, 14), (73, 83), (74, 20), (74, 77), (76, 20), (76, 77), (80, 10), (80, 87), (83, 23), (83, 74), (84, 37), (84, 60), (85, 26), (85, 71), (86, 28), (86, 69), (87, 27), (87, 70), (88, 41), (88, 56), (91, 39), (91, 58), (92, 16), (92, 81), (95, 31), (95, 66), (96, 0)]
Total poi

In [17]:
def Run(point_on_curve):
    private_key, public_key = generate_keypair()
    print(f"Private Key: {private_key}")
    print(f"Public Key: {public_key}")

    C1, C2 = encrypt(point_on_curve, public_key)
    print(f"Encrypted: C1 = {C1}, C2 = {C2}")

    decrypted = decrypt(C1, C2, private_key)
    print(f"Decrypted: {decrypted}")
    
for pt in points_on_curve:
    print(f"Testing point: {pt}")
    Run(pt)

Testing point: (0, 10)
Doubling point: (80, 10)
Doubling point: (3, 91)
Doubling point: (80, 87)
Doubling point: (3, 6)
Doubling point: (80, 10)
Adding None and (80, 10)
Result after addition: (80, 10)
Doubling point: (3, 91)
Adding (80, 10) and (3, 91)
Result after addition: (3, 6)
Doubling point: (80, 87)
Private Key: 96
Public Key: (3, 6)
Doubling point: (80, 10)
Adding None and (80, 10)
Result after addition: (80, 10)
Doubling point: (3, 91)
Adding (80, 10) and (3, 91)
Result after addition: (3, 6)
Doubling point: (80, 87)
Doubling point: (3, 6)
Adding (3, 6) and (3, 6)
Result after addition: (80, 10)
Doubling point: (80, 10)
Doubling point: (80, 10)
Adding None and (80, 10)
Result after addition: (80, 10)
Doubling point: (3, 91)
Adding (80, 10) and (3, 91)
Result after addition: (3, 6)
Doubling point: (80, 87)
Doubling point: (3, 6)
Adding (3, 6) and (3, 6)
Result after addition: (80, 10)
Doubling point: (80, 10)
Encrypted: C1 = (80, 10), C2 = (17, 87)
Doubling point: (3, 91)
Doub

TypeError: 'NoneType' object is not subscriptable